<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/10_modernRNNs_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

* RNNs are faced with two numerical stability problems: exploding and vanishing gradients.
* For exploding gradients, the gradient clipping method was introduced in the previous chapter

## 10.1 Long Short-Term Memory (LSTM)

* Long short-term memory (LSTM) was used to address the vanishing gradients in RNNs.
* Recurrent Neural Networks suffer from short-term memory due to vanishing gradient problem, especially when working with large data sequences.
* Newer versions of RNNs can preserve important information from earlier parts of the sequence and carry it forward.